In [218]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Read data
data_path = "small_data.xlsx"
#data_path = "nurse_schedule_project2_data_large_VA.xlsx"
locations = pd.read_excel(data_path, sheet_name="locations", index_col = 0, header = None)
nurses = pd.read_excel(data_path, sheet_name="nurses", index_col = 0)
task_time = pd.read_excel(data_path, sheet_name="task_execution_time", index_col = 0).T

patients = pd.read_excel(data_path, sheet_name="patients", index_col = 0)
# Break comma separated strings into lists
for col in ['M', 'T', 'W','Th','F','S','Su']:
    patients[col] = patients[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])
for col in ['skillset']:
    nurses[col] = nurses[col].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Transpose the dataframe to access patient information easily
patients = patients.T
nurses = nurses.T


# Sets and Parameters
N = nurses.columns.tolist()            # set of nurses
NursesSkills = nurses.loc['skillset']  # set of skills
P = patients.columns.tolist()          # set of patients
L = list(locations.index)         # set of locations
D = ['M', 'T', 'W', 'Th', 'F', 'S', 'Su']  # set of days
T_tasks = task_time.columns.tolist() # set of tasks

# Parameters


# define a function to list all patients in a given location
def find_keys_with_inner_value(df, target_value):
    return list(df.T[df.T['location'] == target_value].index)

print(find_keys_with_inner_value(patients, 'L1'))

[]


# Code below will make a schedule confirming to the 40 hour limit only

In [219]:
# Initialize a dictionary to hold the schedule
schedule = {nurse: {'M': [], 'T': [], 'W': [], 'Th': [], 'F': [], 'S': [], 'Su': []} for nurse in N}

# Assuming that 'task_time' is a DataFrame where index are task names and there's a 'Time' column.
# Convert task_time to a dictionary for easier use in the function if required.
task_time_dict = task_time.to_dict('index')

# Function to assign nurses to patients based on skillset matching
def assign_nurses_to_patients(nurses, patients, task_time_dict, max_hours_per_day=10):
    # Initialize a schedule dictionary
    schedule = {nurse: {day: [] for day in D} for nurse in nurses.columns}

    # Initialize a dictionary to keep track of the total hours worked by each nurse each day
    nurse_hours = {nurse: {day: 0 for day in D} for nurse in nurses.columns}

    # Initialize binary variables for task assignment (0 = not assigned, 1 = assigned)
    task_assigned = {(patient, task, day): 0 for day in D for patient in P for task in T_tasks}

    # Iterate over the days and patients to assign tasks to nurses based on their skills
    for day in D:  # 'D' is the list of days
        for patient in P:  # 'P' is the list of patient IDs
            # Check if the patient's info for the day is a list indicating they have tasks that day
            if isinstance(patients.loc[day, patient], list):
                patient_tasks = patients.loc[day, patient]
                for task in patient_tasks:
                    for nurse in nurses.columns:
                        nurse_skills = nurses.loc['skillset', nurse]
                        # Access the duration for the task
                        task_duration = task_time_dict['Time'].get(task, 0) / 60  # Convert minutes to hours
                        
                        # Check if the nurse has the skill and the task is not already assigned
                        if task in nurse_skills and nurse_hours[nurse][day] + task_duration <= max_hours_per_day and not task_assigned[(patient, task, day)]:
                            # Assign task to nurse for this day
                            schedule[nurse][day].append((patient, task))
                            # Update hours for the nurse
                            nurse_hours[nurse][day] += task_duration
                            # Mark the task as assigned
                            task_assigned[(patient, task, day)] = 1

    return schedule, nurse_hours

# Call the updated function
updated_schedule, nurse_hours = assign_nurses_to_patients(nurses, patients, task_time_dict)

print(updated_schedule)
print(nurse_hours)


{'Nurse_1': {'M': [], 'T': [], 'W': [], 'Th': [], 'F': [('Patient_1', 'administering injections')], 'S': [('Patient_1', 'administering injections'), ('Patient_3', 'administering injections')], 'Su': [('Patient_1', 'administering injections')]}, 'Nurse_2': {'M': [], 'T': [], 'W': [], 'Th': [], 'F': [], 'S': [], 'Su': [('Patient_2', 'wound care')]}}
{'Nurse_1': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0.4166666666666667, 'S': 0.8333333333333334, 'Su': 0.4166666666666667}, 'Nurse_2': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0.8333333333333334}}


# Code below will check a returned schedule and return a list of nurses who do not work consecutive days

This code is working when tested for non consecutive days including SUNDAY to MONDAY

In [220]:
# Redefining the function with the correct logic to handle Sunday and Monday consecutiveness

def check_consecutive_days(schedule, days):
    # Helper function to determine if two days are consecutive
    def is_consecutive(day1, day2, day_order):
        idx1 = day_order.index(day1)
        idx2 = day_order.index(day2)
        # Check for consecutive days considering wrap around
        return (idx1 + 1) % len(day_order) == idx2 or (idx2 + 1) % len(day_order) == idx1

    # Initialize a dictionary to keep track of patients' schedules
    patient_schedule = {}
    for nurse in schedule:
        for day in schedule[nurse]:
            for appointment in schedule[nurse][day]:
                patient, _ = appointment
                if patient not in patient_schedule:
                    patient_schedule[patient] = []
                patient_schedule[patient].append(day)

    # Check for patients not working on consecutive days
    non_consecutive_patients = []
    for patient in patient_schedule:
        patient_days = patient_schedule[patient]
        # Sort days by the index in the 'days' list to simplify consecutive check
        sorted_days = sorted(patient_days, key=lambda x: days.index(x))
        # Check each day with the next to see if any are consecutive
        consecutive = any(is_consecutive(sorted_days[i], sorted_days[i + 1], days)
                          for i in range(len(sorted_days) - 1))
        # Check for wrap-around case separately (Sunday to Monday)
        if not consecutive and is_consecutive(sorted_days[-1], sorted_days[0], days):
            consecutive = True
        if not consecutive:
            non_consecutive_patients.append(patient)

    return non_consecutive_patients

# Now call the function with the current schedule to find out which nurses have non-consecutive schedules
non_consecutive_schedule_nurses = check_consecutive_days(updated_schedule, D)

print(non_consecutive_schedule_nurses)

['Patient_3', 'Patient_2']


# Code below will return a dict of nurses with an array of days as values. The days are those in which the nurse works at multiple locations

In [221]:
def find_nurses_with_different_locations(schedule, find_keys_with_inner_value, L, patients):
    # Find out which patients are at each location
    patients_at_locations = {location: find_keys_with_inner_value(patients, location) for location in L}

    # Reverse map to see which location each patient is at for quick lookup
    patient_location_map = {}
    for location, location_patients in patients_at_locations.items():
        for patient in location_patients:
            patient_location_map[patient] = location

    # Store nurses who work with patients in more than one location on a single day
    nurses_with_different_locations = {}

    # Check each nurse's schedule
    for nurse, days in schedule.items():
        for day, appointments in days.items():
            locations_worked = set()  # Initialize locations_worked for each day
            for patient, _ in appointments:
                if patient in patient_location_map:  # Check if patient has a mapped location
                    locations_worked.add(patient_location_map[patient])
            # If a nurse works in more than one location in the same day, add to the result
            if len(locations_worked) > 1:
                if nurse not in nurses_with_different_locations:
                    nurses_with_different_locations[nurse] = []
                nurses_with_different_locations[nurse].append(day)

    return nurses_with_different_locations

# Call the function with the current schedule to find out which nurses have different locations
nurses_with_different_locations = find_nurses_with_different_locations(updated_schedule, find_keys_with_inner_value, L, patients)

print(nurses_with_different_locations)
print(updated_schedule)

{'Nurse_1': ['S']}
{'Nurse_1': {'M': [], 'T': [], 'W': [], 'Th': [], 'F': [('Patient_1', 'administering injections')], 'S': [('Patient_1', 'administering injections'), ('Patient_3', 'administering injections')], 'Su': [('Patient_1', 'administering injections')]}, 'Nurse_2': {'M': [], 'T': [], 'W': [], 'Th': [], 'F': [], 'S': [], 'Su': [('Patient_2', 'wound care')]}}


Some kind of a method to check if a task can be reassigned from nurse X to nurse Y without causing conflicts

Some kind of method that will take in a nurse and check if it alone has a valid schedule maybe?

# (ignore below)

In [222]:
# City with needs on each day,
# array of 7 lists, each list contains patients in that city on that day 

# total task time / city / day 

# function to determine total task time / city / day
def calculate_total_hours_per_city_per_day(day, city): 
    total_hours = 0
    for patient in find_keys_with_inner_value(patients, city):
        for task in patients[patient][day]:
            total_hours += task_time[task]
    return total_hours

# function to determine total task time / city / day / task
def calculate_total_hours_per_city_per_day_per_task(day, city, task): 
    total_hours = 0
    for patient in find_keys_with_inner_value(patients, city):
        for theTask in patients[patient][day]:
            if theTask == task:
                total_hours += task_time[task]
    return total_hours


# make an array for each nurse with the total hours they work on each day (init to 0)
eachNursesHours = {}
for nurse in N:
    eachNursesHours[nurse] = {}
    for day in D:
        eachNursesHours[nurse][day] = 0

print(eachNursesHours)

# make an array for each city with the total hours of work needed on each day using the function above
eachCityHours = {}
for city in L:
    eachCityHours[city] = {}
    for day in D:
        eachCityHours[city][day] = {}
        for task in T_tasks:
            eachCityHours[city][day][task] = calculate_total_hours_per_city_per_day_per_task(day, city, task)

print()
print()
print()
print(eachCityHours)

# loop through each city for each day and assign a nurse to each task's total time if the nurse has the skillset and is available
# THIS FUNCTRION DOES NOT WORK YET @JOSH we will fix it later 
def assignNurses():
    for city in L:
        for day in D:
            for task in T_tasks:
                for nurse in N:
                    if task in NursesSkills['Nurse_1'] and eachNursesHours[nurse][day] < eachCityHours[city][day][task]:
                        eachNursesHours[nurse][day] += eachCityHours[city][day][task]
                        print("Nurse " + nurse + " assigned to " + task + " in " + city + " on " + day)
                        break

#assignNurses()


{'Nurse_1': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}, 'Nurse_2': {'M': 0, 'T': 0, 'W': 0, 'Th': 0, 'F': 0, 'S': 0, 'Su': 0}}



{'Alexandria': {'M': {'medication': 0, 'drawing blood': 0, 'physical therapy': 0, 'wound care': 0, 'administering injections': 0, 'personal hygiene assistance': 0}, 'T': {'medication': 0, 'drawing blood': 0, 'physical therapy': 0, 'wound care': 0, 'administering injections': 0, 'personal hygiene assistance': 0}, 'W': {'medication': 0, 'drawing blood': 0, 'physical therapy': 0, 'wound care': 0, 'administering injections': 0, 'personal hygiene assistance': 0}, 'Th': {'medication': 0, 'drawing blood': 0, 'physical therapy': 0, 'wound care': 0, 'administering injections': 0, 'personal hygiene assistance': 0}, 'F': {'medication': 0, 'drawing blood': 0, 'physical therapy': 0, 'wound care': 0, 'administering injections': 0, 'personal hygiene assistance': 0}, 'S': {'medication': 0, 'drawing blood': 0, 'physical therapy': 0, 'wound care': 0, 'administ